# Recommendation system

## Imports and db connection

In [1]:
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
#import streamlit as st
import time
import matplotlib.pyplot as plt

In [2]:
connexion = sqlite3.connect("../database/imdb/imdb.db")
cursor = connexion.cursor()

In [3]:
# creating dataframe for movie details table
cursor.execute("""
    SELECT tb.primaryTitle, tb.startYear, tb.genres, tc.directors, tr.averageRating, tr.numVotes
    FROM title_basics tb, title_crew tc, title_ratings tr 
    ON tb.tconst = tc.tconst AND tb.tconst = tr.tconst
    where tb.isAdult = 0 and tb.titleType = 'movie'
""")
data = cursor.fetchall()
col = [description[0] for description in cursor.description]

df = pd.DataFrame.from_records(data=data, columns=col)

In [4]:
df.shape

(219235, 6)

## Exploratory analysis

In [5]:
df.head()

,primaryTitle,startYear,genres,directors,averageRating,numVotes
0,Miss Jerry,1894,Romance,nm0085156,5.3,200
1,Bohemios,1905,\N,nm0063413,4.2,14
2,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",nm0846879,6.0,797
3,The Prodigal Son,1907,Drama,nm0141150,5.1,20
4,Robbery Under Arms,1907,Drama,nm0533958,4.3,23


In [6]:
# convert data types
df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)

In [7]:
# apply filters to reduce dataset
df_above5 = df[df['averageRating'] >= 8.0]
df_above5

,primaryTitle,startYear,genres,directors,averageRating,numVotes
150,Captain Alvarez,1914,Drama,nm0836316,8.0,17
192,The Man from Mexico,1914,Comedy,nm0373614,8.5,39
245,1915 World's Championship Series,1915,"Documentary,Sport",\N,8.4,20
283,The Devil,1915,Drama,"nm0054977,nm0408436",8.3,18
285,The Dictator,1915,"Adventure,Romance",nm0247107,8.0,45
...,...,...,...,...,...,...
219210,Dosage Volume I,2002,"Action,Documentary,Sport",nm1355731,8.3,7
219213,Your Iron Lady,2020,Drama,nm5609795,8.6,15
219222,Ott Tänak: The Movie,2019,"Documentary,Sport",nm4942142,8.1,488
219224,Pengalila,2019,Drama,nm0151535,8.0,678


In [8]:
# apply filters to reduce dataset
df_clean = df_above5[df_above5['numVotes'] >= 1000]
df_clean

,primaryTitle,startYear,genres,directors,averageRating,numVotes
995,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,8.0,64645
1407,The Kid,1921,"Comedy,Drama,Family",nm0000122,8.3,126941
1412,The Phantom Carriage,1921,"Drama,Fantasy,Horror",nm0803705,8.0,12789
1872,Safety Last!,1923,"Action,Comedy,Thriller","nm0628345,nm0853130",8.1,20981
2029,The Last Laugh,1924,Drama,nm0003638,8.0,14221
...,...,...,...,...,...,...
218649,Brochevarevarura,2019,Comedy,nm9445831,8.0,4145
218650,Mallesham,2019,Biography,nm10459557,8.3,1074
218981,The Bee Gees: How Can You Mend a Broken Heart,2020,"Biography,Documentary,Music",nm0550881,8.1,4914
218988,Major,2022,"Action,Biography,Drama",nm3481620,8.2,27793


In [9]:
# convert data types
df_clean['genres'] = df_clean['genres'].astype("string")
df_clean['directors'] = df_clean['directors'].astype("string")

/tmp/ipykernel_11674/1430223804.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['genres'] = df_clean['genres'].astype("string")
/tmp/ipykernel_11674/1430223804.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['directors'] = df_clean['directors'].astype("string")


In [10]:
df_clean.dtypes

primaryTitle      object
startYear         object
genres            string
directors         string
averageRating    float64
numVotes           int64
dtype: object

In [11]:
df_clean.head()

,primaryTitle,startYear,genres,directors,averageRating,numVotes
995,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,8.0,64645
1407,The Kid,1921,"Comedy,Drama,Family",nm0000122,8.3,126941
1412,The Phantom Carriage,1921,"Drama,Fantasy,Horror",nm0803705,8.0,12789
1872,Safety Last!,1923,"Action,Comedy,Thriller","nm0628345,nm0853130",8.1,20981
2029,The Last Laugh,1924,Drama,nm0003638,8.0,14221


## Data processing

In [12]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [13]:
# Apply clean_data function to your features.
features = ['directors', 'genres']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)


/tmp/ipykernel_11674/952854994.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[feature] = df_clean[feature].apply(clean_data)


In [14]:
def create_soup(x):
    return ' '.join(x['directors']) + ' ,' + ' '.join(x['genres'])

In [15]:
# Create a new soup column representing the features
df_clean['soup'] = df_clean.apply(create_soup, axis=1)


/tmp/ipykernel_11674/2773241301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['soup'] = df_clean.apply(create_soup, axis=1)


In [16]:
df_clean['soup'].head()
print(df_clean['soup'])

995       n m 0 9 2 7 4 6 8 ,h o r r o r , m y s t e r y...
1407      n m 0 0 0 0 1 2 2 ,c o m e d y , d r a m a , f...
1412      n m 0 8 0 3 7 0 5 ,d r a m a , f a n t a s y ,...
1872      n m 0 6 2 8 3 4 5 , n m 0 8 5 3 1 3 0 ,a c t i...
2029                           n m 0 0 0 3 6 3 8 ,d r a m a
                                ...                        
218649                       n m 9 4 4 5 8 3 1 ,c o m e d y
218650               n m 1 0 4 5 9 5 5 7 ,b i o g r a p h y
218981    n m 0 5 5 0 8 8 1 ,b i o g r a p h y , d o c u...
218988    n m 3 4 8 1 6 2 0 ,a c t i o n , b i o g r a p...
219204    n m 7 9 9 2 2 3 1 ,a c t i o n , a d v e n t u...
Name: soup, Length: 1307, dtype: object


In [17]:
# clean the soup column
features = ['soup']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)

/tmp/ipykernel_11674/105095674.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[feature] = df_clean[feature].apply(clean_data)


In [18]:
print(df_clean['soup'])

995                nm0927468,horror,mystery,thriller
1407                   nm0000122,comedy,drama,family
1412                  nm0803705,drama,fantasy,horror
1872      nm0628345,nm0853130,action,comedy,thriller
2029                                 nm0003638,drama
                             ...                    
218649                              nm9445831,comedy
218650                          nm10459557,biography
218981         nm0550881,biography,documentary,music
218988              nm3481620,action,biography,drama
219204              nm7992231,action,adventure,crime
Name: soup, Length: 1307, dtype: object


In [19]:
# Function to convert all strings to lower case and strip names of spaces
df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)

/tmp/ipykernel_11674/2337589423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)


In [20]:
df_clean.head()

,primaryTitle,startYear,genres,directors,averageRating,numVotes,soup
995,The Cabinet of Dr. Caligari,1920,"horror,mystery,thriller",nm0927468,8.0,64645,nm0927468 horror mystery thriller
1407,The Kid,1921,"comedy,drama,family",nm0000122,8.3,126941,nm0000122 comedy drama family
1412,The Phantom Carriage,1921,"drama,fantasy,horror",nm0803705,8.0,12789,nm0803705 drama fantasy horror
1872,Safety Last!,1923,"action,comedy,thriller","nm0628345,nm0853130",8.1,20981,nm0628345 nm0853130 action comedy thriller
2029,The Last Laugh,1924,drama,nm0003638,8.0,14221,nm0003638 drama


In [21]:
# convert data types
df_clean['primaryTitle'] = df_clean['primaryTitle'].astype("string")

/tmp/ipykernel_11674/291099518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['primaryTitle'] = df_clean['primaryTitle'].astype("string")


In [22]:
df_clean = df_clean.reset_index()

In [23]:
df_clean.dtypes

index              int64
primaryTitle      string
startYear         object
genres            object
directors         object
averageRating    float64
numVotes           int64
soup              object
dtype: object

## Create 1 vector for each movie and calculate their similarities

In [24]:
# Import CountVectorizer and one vector per movie for the soup column
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_clean['soup'])

In [25]:
count_matrix[0]

<1x1148 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [26]:
# 

In [27]:
# Compute the Cosine Similarity matrix for every vector
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


In [28]:
type(cosine_sim2)

numpy.ndarray

In [29]:
# Function that takes in a list of movie titles as input and outputs 10 most similar movies
def get_recommendations_list(liked_movies_list, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    
    idx_list=[]
    for m in liked_movies_list:
        
        idx = df_clean.index[df_clean['primaryTitle'] == m]
        idx_list.append(idx)

    df_sim_list=pd.DataFrame()
    print(idx_list)
    for m in idx_list:
        sim_scores = cosine_sim2[m]
        df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=[m])
        df_sim_list[m]=df_sim

    df_sim_list['average']=df_sim_list.mean(numeric_only=True, axis=1)

    rslt_df = df_sim_list.sort_values(by='average',ascending=False)

    # print(rslt_df[0:11])
    rec_list=rslt_df.reset_index()
    list1=[]
    list1=rec_list['index'][0:11].values

    # Return the top 10 most similar movies
    return df_clean['primaryTitle'].iloc[list1]


In [31]:
liked_movies_list=['The Last Laugh','Safety Last!','The Kid']

recommendations = get_recommendations_list(liked_movies_list)
print(recommendations)

[Int64Index([4], dtype='int64'), Int64Index([3], dtype='int64'), Int64Index([1], dtype='int64')]
1                       The Kid
4                The Last Laugh
595                  Happy Days
570                  Our Family
37           The Great Dictator
454               Laughing Eyes
27                 Modern Times
22                  City Lights
830            Ratnan Prapancha
87      The Kingdom of Diamonds
1057            #Kaadal Kahaani
Name: primaryTitle, dtype: string
